In [ ]:
#use new network 
#use the past strategy

In [1]:
from collections import OrderedDict

import torch
from torch import nn


class IntermediateLayerGetter(nn.ModuleDict):
    """
    Module wrapper that returns intermediate layers from a model
    It has a strong assumption that the modules have been registered
    into the model in the same order as they are used.
    This means that one should **not** reuse the same nn.Module
    twice in the forward if you want this to work.
    Additionally, it is only able to query submodules that are directly
    assigned to the model. So if `model` is passed, `model.feature1` can
    be returned, but not `model.feature1.layer2`.
    Arguments:
        model (nn.Module): model on which we will extract the features
        return_layers (Dict[name, new_name]): a dict containing the names
            of the modules for which the activations will be returned as
            the key of the dict, and the value of the dict is the name
            of the returned activation (which the user can specify).
    Examples::
        >>> m = torchvision.models.resnet18(pretrained=True)
        >>> # extract layer1 and layer3, giving as names `feat1` and feat2`
        >>> new_m = torchvision.models._utils.IntermediateLayerGetter(m,
        >>>     {'layer1': 'feat1', 'layer3': 'feat2'})
        >>> out = new_m(torch.rand(1, 3, 224, 224))
        >>> print([(k, v.shape) for k, v in out.items()])
        >>>     [('feat1', torch.Size([1, 64, 56, 56])),
        >>>      ('feat2', torch.Size([1, 256, 14, 14]))]
    """
    def __init__(self, model, return_layers):
        if not set(return_layers).issubset([name for name, _ in model.named_children()]):
            raise ValueError("return_layers are not present in model")

        orig_return_layers = return_layers
        return_layers = {k: v for k, v in return_layers.items()}
        layers = OrderedDict()
        for name, module in model.named_children():
            layers[name] = module
            if name in return_layers:
                del return_layers[name]
            if not return_layers:
                break

        super(IntermediateLayerGetter, self).__init__(layers)
        self.return_layers = orig_return_layers
        #self.softmax = nn.Softmax(dim=2)

    def forward(self, x):
        out = OrderedDict()
        for name, module in self.named_children():
#             print(name)
            out[name] = module(x)
            x= out[name]
#             print(x.shape)
            if name == 'ada_maxpool':
                x=torch.mean(x,(1))
                x=x.view(x.size(0))
            if name == 'sigmoid':
                x = out['relu']
            if name =='layer4':
                break
        return out

In [2]:
import torch.nn as nn
#from .utils import load_state_dict_from_url



def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet2(nn.Module):

    def __init__(self, block, layers, num_classes=2, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet2, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        self.inplanes2 = 1024
        self.dilation2 = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(1, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
#         self.avgpool = nn.AvgPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer11 = self._make_layer(block, 64, 3,flag=True)
        self.layer22 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer33 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.ada_maxpool = nn.AdaptiveMaxPool2d((1, 1))
        self.sigmoid=nn.Sigmoid()
        
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer2(block, 64, layers[0])

        self.layer2 = self._make_layer2(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer2(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer2(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
#         self.ada_avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)
        self.fc2 = nn.Linear(2, 1)
        self.softmax=nn.Softmax(dim=1)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False,flag=False):
        if flag:
            self.dilation2=self.dilation
            self.inplanes2=self.inplanes
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)
    def _make_layer2(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation2 = self.dilation2
        if dilate:
            self.dilation2 *= stride
            stride = 1
        if stride != 1 or self.inplanes2 != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes2, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes2, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation2, norm_layer))
        self.inplanes2 = planes * block.expansion

        for _ in range(1, blocks):
            layers.append(block(self.inplanes2, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def forward(self, x):
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
#         self.inplanes2 = self.inplanes
#         self.dilation2 = self.dilation
        
        y = self.maxpool(x)
#         x = self.avgpool(x)
#         self.inplanes2 = self.inplanes
#         self.dilation2 = self.dilation
#         print(x.shape)
#         print(y.shape)
#         print(self.inplanes,self.dilation,self.inplanes2,self.dilation2)
#         y = self.layer1(y)
        x = self.layer11(x)
        y = self.layer1(y)
        y = self.layer2(y)
        x = self.layer22(x)
        y = self.layer3(y)
        x = self.layer33(x)
        y = self.layer4(y)
        x = self.ada_maxpool(x)#所以这里都是均值 
        y = self.ada_avgpool(y)
        y = y.reshape(y.size(0), -1)
        y = self.fc(y)
        y = self.softmax(y)
        
        x = torch.mean(x,(1))
        
#         print("fdkjsfds")
#         print(x.shape)
        x = x.view(x.size(0))
        x = self.sigmoid(x)
# #         print(x.item())
# #         print(y.item())
# #         x = x*1+y*0
# #         print(x.item())
# #         print(x.item())
# #         print(x.shape)
# #         print(y.shape)
# #         print("fdkjlsf")
        out = OrderedDict()
        out['x'] = x
        out['y'] = y
        
        return out


def _resnet(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet2(block, layers, **kwargs)
    return model



def resnet101_(pretrained=False, progress=True, **kwargs):
    r"""ResNet-101 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>'_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet101', Bottleneck, [3, 4, 9, 3], pretrained, progress,
                   **kwargs)


In [3]:
import numpy as np
import torch
from torchvision.models import resnet101
import torch.nn as nn
import os
import cv2
import random
import torch.utils.model_zoo as model_zoo
from copy import deepcopy

from matplotlib import cm
import scipy.misc as misc
import matplotlib
import imageio

def get_jet():
    colormap_int = np.zeros((256, 3), np.uint8)
    colormap_float = np.zeros((256, 3), np.float)

    for i in range(0, 256, 1):
        colormap_float[255-i, 0] = cm.jet(i)[0]
        colormap_float[255-i, 1] = cm.jet(i)[1]
        colormap_float[255-i, 2] = cm.jet(i)[2]

        colormap_int[255-i, 0] = np.int_(np.round(cm.jet(i)[0] * 255.0))
        colormap_int[255-i, 1] = np.int_(np.round(cm.jet(i)[1] * 255.0))
        colormap_int[255-i, 2] = np.int_(np.round(cm.jet(i)[2] * 255.0))
        #colormap_int = colormap_int[::-1]
    return colormap_int




class ban_tanh(nn.Module):
    def forward(self, x):
        x=torch.clamp(x,min=0)
        return torch.tanh(x)

# def load_data(img_path,label_path):
#     labels_list=open(label_path, 'r')
#     X,Y,Z=[],[],[]
#     i=1
#     for line in labels_list:
#         info=str.split(line,",") # info[1]label； info[2]图片文件名
#         X.append(np.concatenate([deepcopy (np.expand_dims(imageio.imread(info[1]),axis=2))for i in range(3)],axis=2))
#         Y.append(int(info[0]))
#         Z.append(str(info[2]))
#         i+=1
#         if i%100==0:
#             print(i)
# #         print(X)
# #         break;
#     return list(zip(X,Y,Z))

class FeatureExtractor(nn.Module):
    def __init__(self, submodule, extracted_layers):
        super(FeatureExtractor, self).__init__()
        self.submodule = submodule
        self.extracted_layers = extracted_layers

    def forward(self, x):
        outputs = []
        _name = []
        for name, module in self.submodule._modules.items():
            if name is "fc": x = x.view(x.size(0), -1)
            #print(name)
            _name.append(name)
            x = module(x)

            #if name in self.extracted_layers:
            outputs.append(x)
        return list(zip(_name,outputs))
class FeatureExtractor2(nn.Module):
    def __init__(self, submodule, extracted_layers):
        super(FeatureExtractor2, self).__init__()
        self.submodule = submodule
        self.extracted_layers = extracted_layers

    def forward(self, x):
        outputs = []
        for name, module in self.submodule._modules.items():
            if name is "fc": x = x.view(x.size(0), -1)
           # print(name)
            x = module(x)

            if name in self.extracted_layers:
                outputs.append(x)
        return outputs
    
def normalize_255_uchar(array):
    Max=np.max(array)
    Min=np.min(array)
    return (((array-Min)/(Max-Min))*255).astype(np.uint8)
learning_rate=0.0001

model=resnet101_().cuda()
#model=cam_model()
# criterion = nn.CrossEntropyLoss()
#criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
class_num = 2
channel_in = model.fc.in_features
model.fc = nn.Linear(channel_in,class_num).cuda()#注意加上这个cuda()


In [4]:
# print(model)

In [5]:
import csv,os


In [6]:
#you should always put the image at the second place 
import csv,os
def get_csv():
    class_path  =  'F:\\Ruiveen2\\Data\\Train_c1\\expert1\\'
    class_path3  = 'F:\\Ruiveen2\\Data\\img\\'
    blanket=set()
    #############case3
#     blanket.add(6)
#     blanket.add(21)
#     blanket.add(27)
#################case1
    blanket.add(8)
    blanket.add(9)
    blanket.add(2)
###############case2
#     blanket.add(5)
#     blanket.add(15)
#     blanket.add(18)
###############case4
#     blanket.add(7)
#     blanket.add(29)
#     blanket.add(34)

#     if 2 in blanket:
#         print("fdsfdsfds")
    with open('my_train.csv','w',newline='')as f:
        f_csv = csv.writer(f)
        i=0
        for img_name in os.listdir(class_path):
            info=str.split(img_name,"_")
            
#             print(img_name)
#             break
           # print(info[0])
            if int(info[0]) not in blanket:
                class_path2=class_path+img_name+"\\"
#                 print(class_path2)
#                 break
                for img_name2 in os.listdir(class_path2):
                    img_name3=info[1]+"_"+img_name2
                   # print(img_name3)
                    img_path2= class_path2+ img_name2
                    img_path = class_path3+info[1]+"\\"+ img_name2
#                     print(img_path)
#                     print(img_path2)
#                     break
                    temp=[2]
                    temp.append(img_path)
                    temp.append(img_name3)
                    temp.append(img_path2)
                    temp.append("")
                    f_csv.writerow(temp)
                    i=i+1
                    if i%100==0:
                        print(i)
#                     break
             #   break
get_csv()


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900


In [7]:
#get_csv()

In [8]:
def load_data(label_path):
    labels_list=open(label_path, 'r')
    X,Y,Z=[],[],[]
    i=1
    for line in labels_list:
        info=str.split(line,",") 
        if i==1:
            print(info)
        X.append(np.expand_dims(imageio.imread(info[1]),axis=2))
        a=imageio.imread(info[3])
        b=np.max(a)
        if b!=0:
            b=1
        Y.append(b)
        Z.append(str(info[2]))
        #break
#         print(X[0].shape)
#         print(X[0].shape)
        i+=1
        if i%100==0:
            print(i)
#             if i==500:
#                 break
    return list(zip(X,Y,Z))

In [9]:
data=load_data("my_train.csv")

['2', 'F:\\Ruiveen2\\Data\\img\\17618\\1.bmp', '17618_1.bmp', 'F:\\Ruiveen2\\Data\\Train_c1\\expert1\\15_17618\\1.bmp', '\n']
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900


In [10]:
model.load_state_dict(torch.load("com_heat.pkl"))
return_layers = {'layer4': 'layer4'}
model=IntermediateLayerGetter(model, return_layers=return_layers)

In [12]:
mmpp=87685435454

In [13]:
def get_G(outputs,list1,_,flag):# get the middle image
    for k,v in outputs.items():
        temp = np.squeeze(v.cpu().numpy())
#         if k=='ada_maxpool':
#             temp2=np.max(temp)
#             print(temp2)
#             list1['layer33_num']=temp2
        if k=='sigmoid' or k =='ada_maxpool':
            list1[k]=temp
            continue
        temp=normalize_255_uchar(np.sum(temp,axis=0)).astype(np.int)
#         print(temp.shape)
        c=temp.shape[0]
        d=temp.shape[1]
        hp=np.zeros((c,d,3),np.uint8)
        for i in range(c):
            for j in range(d):
                hp[i][j]=color_map[temp[i][j]]
#         print(_)
        if flag:
            cv2.imwrite("val_tanh_bce_nofc_c8/" +_+"_"+k+".bmp", hp)
        list1[k]=temp
#         print (k,v.shape)

In [14]:
def layer_gain3(v3,sf2,m,n,o,tp):
#     v3=torch.sum(v3,dim=1)
#     v3=np.squeeze(n_255(v3).cpu().numpy().astype(np.uint8))
    c=v3.shape[0]
    d=v3.shape[1]
    sf2 = (cv2.resize(sf2, dsize=(d, c)))
    ret,sf3=cv2.threshold(sf2,200,255,cv2.THRESH_BINARY)
    if np.max(sf3)==0:
        ret,sf2=cv2.threshold(sf2,170,255,cv2.THRESH_BINARY)
    else :
        sf2=sf3
    thres_=-9999
    for i in range(c):
        for j in range(d):
            if sf2[i][j]==255:
                thres_=max(thres_,v3[i][j])
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    for k in range(m):
        sf2 = cv2.dilate(sf2, kernel)
        for i in range(c):
            for j in range(d):
                if v3[i][j]>thres_:
                    sf2[i][j]=0
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (o, 5))
    if tp==0:
        for k in range(n):
            sf2 = cv2.dilate(sf2, kernel)
            for i in range(c):
                for j in range(d):
                    if sf2[i][j]>0:
                        if v3[i][j]>thres_:
                            sf2[i][j]=0
    else:
        lm=9999999
        rm=-9899999
        
        for k in range(n):
            sf2 = cv2.dilate(sf2, kernel)
            for i in range(c):
                for j in range(d):
                    if sf2[i][j]>0:
                        if v3[i][j]>thres_:
                            sf2[i][j]=0
                        else:
                            lm=min(lm,j)
                            rm=max(rm,j)
        sf=np.sum(sf2/255,axis=0)
        sf3=np.zeros(128,np.uint8)
        for i in range(0,sf.shape[0]-2):
            if sf[i]>2:
                sf3[i]=abs(sf[i+1]-sf[i])
       # print(sf3)
        a=np.argmax(sf3)
        if sf3[a]>10 and sf[0]>9:
            for i in range(c):
                for j in range(0,a+3):
                    sf2[i][j]=0
        else :
            for k in range(n):
                lt=9999999
                rt=-9899999
                sf2 = cv2.dilate(sf2, kernel)
                for i in range(c):
                    for j in range(d):
                        if sf2[i][j]>0:
                            if v3[i][j]>thres_:
                                sf2[i][j]=0
                            else:
                                lt=min(lt,j)
                                rt=max(rt,j)
                if lt>=lm or rt<=rm:
                    break

        print(lm,rm)
    return sf2,thres_

In [15]:
blanket=set()
cck=[]
pl=np.zeros(1024).astype(np.int)
pr=np.zeros(1024).astype(np.int)
def _sum(u,x,y):
    t=0
#     if x+2>=1024||y+2>=512:
#         return 456654564564
#     print(x,y)
    for i in range(x-1,x+2):
        for j in range(y-1,y+2):
            t+=u[i][j]
    return t/9.0
def _change(u,x,y):
    for i in range(x-1,x+2):
        for j in range(y-1,y+2):
            u[i][j]=255
def _change2(x,y):
    ep=[(x-1,y-1),(x,y-1),(x+1,y-1)]
    for i,j in ep:
        if pl[i]==0:
            pl[i]=j
        else:
            pl[i]=min(pl[i],j)
    ep=[(x-1,y+1),(x,y+1),(x+1,y+1)]
    for i,j in ep:
        if pr[i]==0:
            pr[i]=j
        else:
            pr[i]=max(pr[i],j)
def _gain(g):
    for i in range(pl.shape[0]):
        if pl[i]!=0 and pr[i]>pl[i]:
                col=150
                mid=int((pl[i]+pr[i])/2.0)
                for j in range(pl[i],mid+1):
                    if col<230:
                        col+=1
                    g[i][j]=col
                col=150
                for j in range(pr[i],mid,-1):
                    if col<230:
                        col+=1
                    g[i][j]=col
def _gain2(g):
    for i in range(pl.shape[0]):
        if pl[i]!=0 and pr[i]>pl[i]:
                for j in range(pl[i],pr[i]):
                    g[i][j]=255
def _get_f(f,g):
    for i in range(f.shape[0]):
        for j in range(f.shape[1]):
            g[i][j]=f[i][j]

In [16]:
def feature_search2(feature_sum,thred,feature_sum3,layer):
    c=feature_sum.shape[0]
    d=feature_sum.shape[1]
    print(c,d)
    print(feature_sum3.shape)
    feature_sum3 = (cv2.resize(feature_sum3, dsize=(d,c)))
    global blanket1
    global blanket2
    blanket1=set()
    blanket2=set()
    list1=[]
    sf=np.zeros((c,d),np.uint8)
   # print(np.max(feature_sum3))
    energy=10
    e1=np.max(feature_sum3)
  #  print(e1)
    #if layer==0:
    energy=e1-10
    for i in range(c):
        for j in range(d):
            if feature_sum3[i][j]>energy:
                blanket1.add(i*c+j);
                cc=[(i+4,j),(i-4,j),(i,j-7),(i,j+7)]
                for (xi,xj) in cc:
                    if xi>=c or xj>=d:
                        break
                    list1.append(xi*c+xj)
            elif feature_sum[i][j]>thred:
                sf[i][j]=0
            else:
                sf[i][j]=255
    te=False
    heatmap2=np.zeros((c,d,3),np.uint8)
    for i in range(c):
        for j in range(d):
            if feature_sum3[i][j]>energy:
                heatmap2[i][j]=color_map[150]
            else: 
                heatmap2[i][j]=color_map[sf[i][j]]
    cv2.imwrite("val_tanh_bce_nofc_c8/" +"fdsfds.bmp", heatmap2)
    for k in blanket1:
        if k in blanket2:
            continue
        if dfs2(sf,k,0,layer)==1:
            te=True
            break
#     print(te)
#     print(len(blanket2))
    if len(blanket2)==0:
        for k in list1:
            if k in blanket2:
                continue
            if dfs2(sf,k,0,layer)==1:
                te=True
                break
    elif len(blanket2)<20:
        kernel=cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))
        sq = cv2.dilate(sf, kernel)
        blanket2.clear()
        for k in list1:
            if k in blanket2:
                continue
            if dfs2(sq,k,0,layer)==1:
                te=True
                break
        if len(blanket2)/20<5 and te==False:
            print(len(blanket2)/20)
            blanket2.clear()
            for k in blanket1:
                if k in blanket2:
                    continue
                if dfs2(sf,k,0,layer)==1:
                    te=True
                    break
    print(len(blanket2))
    kkqq=False
    if len(blanket2)>999:
        print("error1")
        kkqq=True
        te=False
        blanket2.clear()
        kernel=cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))
        sf = cv2.erode(sf, kernel)
        for k in list1:
            if k in blanket2:
                continue
            if dfs2(sf,k,0,layer)==1:
                te=True
                break
        if te:
            blanket2.clear()
    print(len(blanket2))
    return sf,kkqq

In [17]:
global blanket1
global blanket2
global blanket3
blanket1=set()
blanket2=set()
blanket2=set()


In [18]:
def dfs2(sf,k,dep,layer):#sf 搜索图 f填充的特征图,s哈希
    c=sf.shape[0]
    d=sf.shape[1]
    if dep>1000:
        return 1
    xi=int(k/c)
    xj=k%c
    e=[]
    if layer==2:
        if dep!=0:
            e=[(xi,xj+1),(xi,xj-1),(xi+1,xj),(xi-1,xj)]
        else:
            e=[(xi,xj+3),(xi,xj-3),(xi+3,xj),(xi-3,xj)]
    elif layer==1 or layer==0:
        e=[(xi,xj+1),(xi,xj-1),(xi+1,xj),(xi-1,xj)]
    for (i,j) in e:
        if i>=c or j>=d:
            return 
        if sf[i][j]==0:
            continue
        k=i*c+j
        if k in blanket2:
            continue
        blanket2.add(k)
        te=dfs2(sf,k,dep+1,layer)
        if te==1:
            return 1
    return 0

In [19]:
def n_255(array):
    Max=torch.max(array)
    Min=torch.min(array)
    return (((array-Min)/(Max-Min))*255)

In [20]:
#test set
t_lay=[]   #get the middle image
t_r=[]
import matplotlib.pyplot as plt # 比较完整的生成膨胀后的图
import copy
with torch.no_grad():
    global blanket1
    global blanket2
    k = 0
    #fft=False
    print(len(data))
    cnt=0
    for img, tar,_ in data:
#         if _!="15382_100.bmp":
#             continue
        print("-----------------------------------")
        print(cnt)
        cnt+=1
#         if fft==False :
#         if _ !="9757_39.bmp":
#             continue
#             else :
#                 fft=True
#         if _!="120.1.bmp":
#             continue
#         if _!="17618_120.bmp":
#             continue
        tk=img
        img=np.concatenate([deepcopy (np.expand_dims(img,axis=2))for i in range(3)],axis=2)#.transpose(3,2, 0, 1)
        img = np.squeeze(img)#.transpose((1, 2, 0))
#             print(img.shape)
#             break
        #print(te.shape)
        tk = np.expand_dims(tk.transpose((2, 0, 1)), 0)
        #print(tk.shape)
       # img = torch.from_numpy(img).cuda().float()
        tk = torch.from_numpy(tk).cuda().float()
#         print(soft_max)
#         print(label)
#         print(_)
        color_map=get_jet()
      #  data2=layers_features(img)
        outputs=model(tk)
        list_m={}
        cnt+=1
        get_G(outputs,list_m,_,0)
        print(tar)
        tar=list_m['sigmoid']
        print(tar)
        if tar<0.9:
#             continue
            sf2=np.zeros((1024,512),np.uint8)
            cv2.imwrite("val_tanh_bce_nofc_c8/"+"_" +_, sf2)
            continue
        else:
#             continue
            feature_sum3=list_m['layer33']#normalize_255_uchar(np.sum(layer3_feature,axis=0)).astype(np.int)#有种子点
            feature_sum=list_m['layer2']
            e=np.mean(feature_sum3)
            print(_)
            c=feature_sum.shape[0]
            d=feature_sum.shape[1]
            heatmap=np.zeros((c,d,3),np.uint8)
            sf2=np.zeros((c,d),np.uint8)
#             img = np.squeeze(img.data.cpu().numpy()).transpose((1, 2, 0))
            tk = np.squeeze(tk.data.cpu().numpy())
#             listt1={}
#             listt2={}
#             get_G(outputs2,outputs,listt1,listt2)
            
#             for k,v in listt1.items():
#                 print (k,v.shape)
#             print("outkdfkjslkjdfslkj2")
#             for k,v in outputs.items():
#                 temp = np.squeeze(v.cpu().numpy())
#                 temp=normalize_255_uchar(np.sum(temp,axis=0)).astype(np.int)
#                 c=temp.shape[0]
#                 d=temp.shape[1]
#                 hp=np.zeros((c,d,3),np.uint8)
#                 for i in range(c):
#                     for j in range(d):
#                         hp[i][j]=color_map[temp[i][j]]
#                 cv2.imwrite("val_tanh_bce_nofc_c8/" +"cnn_"+k+".bmp", hp)
#                 print (k,v.shape)
#             print("outkdfkjslkjdfslkj")
#                 break

            if 0 :
                ;

            else:
                g=np.zeros((c,d),np.uint8)
                _get_f(feature_sum3,g)
#                 e1=np.median(feature_sum)
                e1=np.max(feature_sum, axis=1)
            #     print(temp2.shape)
            #     print(temp2)
#                 e1=np.min(e1)
                e1.sort()
                e1=e1[5]
#                 e1*=2
                sf,kkqq=feature_search2(feature_sum,e1,g,2)
                tp=""
                if len(blanket2)<8 and kkqq==False:
                    layer4_feature = np.squeeze(outputs['layer1'].cpu().numpy())
                    feature_sum=normalize_255_uchar(np.sum(layer4_feature,axis=0)).astype(np.int)
                    c=feature_sum.shape[0]
                    d=feature_sum.shape[1]
                    heatmap=np.zeros((c,d,3),np.uint8)
#                     e1=np.median(feature_sum)
#                     e1*=2
                    e1=np.max(feature_sum, axis=1)
#                     e1=np.min(e1)
                    e1.sort()
                    e1=e1[5]
                    sf,kkqq=feature_search2(feature_sum,e1,g,1)
                    sf2=np.zeros((c,d),np.uint8)
                    tp="001."
                if len(blanket2)<8 and kkqq==False:
                    layer4_feature = np.squeeze(outputs['relu'].cpu().numpy())
                    feature_sum=normalize_255_uchar(np.sum(layer4_feature,axis=0)).astype(np.int)
                    c=feature_sum.shape[0]
                    d=feature_sum.shape[1]
                    heatmap=np.zeros((c,d,3),np.uint8)
                    e1=np.max(feature_sum, axis=1)
                    e1.sort()
                    e1=e1[5]
                    sf,kkqq=feature_search2(feature_sum,e1,g,0)
                    sf2=np.zeros((c,d),np.uint8)
                    tp="000."
                if len(blanket2)<8 and kkqq==False:
                    continue
                    print("AAAAA")
                    tp="A"
                    blanket.clear()
                    cck=[]
                    pl=np.zeros(1024).astype(np.int)
                    pr=np.zeros(1024).astype(np.int)
                    f=img.shape[0]/feature_sum3.shape[0]
                    c=feature_sum3.shape[0]
                    d=feature_sum3.shape[1]
                   # sf2=np.zeros((c,d),np.uint8)
                    g=np.zeros((c,d),np.uint8)
                    _get_f(feature_sum3,g)
                    c=img.shape[0]
                    d=img.shape[1]
                    g=cv2.resize(g, dsize=(d,c))
                    #maxi,maxj= np.where(g==np.max(g))
                    #print(maxi,maxj)
                    h = tk
                    #for (mi,mj) in zip(maxi,maxj):
                    for i in range(c):
                        for j in range(d):
                            if g[i][j]>30:
                                w=ppq(h,g,str(int(i))+str('+')+str(int(j)),0)
    #                 sf2=np.zeros((c,d),np.uint8)
                    _gain2(g)
                    c=feature_sum.shape[0]
                    d=feature_sum.shape[1]
                    #cv2.imwrite("val_tanh_bce_nofc_c2/" +"ooooods.bmp", g)
                    g=cv2.resize(g, dsize=(d,c))
                    cv2.imwrite("val_tanh_bce_nofc_c8/" +"ooooods.bmp", g)
                    sf2=np.zeros((c,d),np.uint8)
                    print(sf2.shape)
                    print(g.shape)
                    print(feature_sum3.shape)
                    for i in range(c):
                        for j in range(d):
                          #  print(sf2[i][j])]
                            if g[i][j]>200:
                                sf2[i][j]=255
                            elif g[i][j]>100:
                                sf2[i][j]=255
                    c=tk.shape[0]
                    d=tk.shape[1]
                    sf3 = (cv2.resize(sf2, dsize=(d,c)))
                    ret,sf3=cv2.threshold(sf3,200,255,cv2.THRESH_BINARY)
                    cv2.imwrite("val_tanh_bce_nofc_c8/"+"_" +_, sf3)
#                     break
                  #  heatmap=np.zeros((c,d,3),np.uint8)
                  #  continue
                elif kkqq==False :
                    #continue
                    print("BBBBB")
    #                 c=feature_sum.shape[0]
    #                 d=feature_sum.shape[1]
    #                 g=cv2.resize(g, dsize=(d,c))
                    for k in blanket2:
                        i=int(k/c)
                        j=k%c
                        if i>=c or j>=d:
                            continue
                        sf2[i][j]=255
                   # print(np.max(sf2))
    #                 for k,v in outputs.items():
    #                     print(k)
    #                     print(v.shape)
#                     print("--0099--")
#                     print(sf2.shape)
#                     get_G2(sf2,"layer2","merge_")
#                     get_G3(sf2,listt1['layer2'],"layer2","show_")
                    
#                     print("layer2")
                    if tp=="":
                        v3=outputs['layer1']
                        v3=torch.sum(v3,dim=1)
                        v3=np.squeeze(n_255(v3).cpu().numpy().astype(np.uint8))
                        if len(blanket2)<20:
                            sf2,res=layer_gain3(v3,sf2,2,8,8,0)
                        else :
                            sf2,res=layer_gain3(v3,sf2,2,4,5,1)
#                     print("--0099--")
#                     get_G2(sf2,"layer1","merge_")
#                     get_G3(sf2,listt1['layer1'],"layer1","show_")
#                     print("layer1")
                  #  print(np.max(sf2))
    #                 print(sf2.shape,v3.shape)
    #                 c=v3.shape[0]
    #                 d=v3.shape[1]
    #                 heatmap=np.zeros((c,d,3),np.uint8)
    #                 for i in range(c):
    #                     for j in range(d):
    #                         if sf2[i][j]==255:
    #                             heatmap[i][j]=color_map[sf2[i][j]]
    #                         else :
    #                             heatmap[i][j]=color_map[(v3[i][j])]
    #                 cv2.imwrite("val_tanh_bce_nofc_c2/" +"B"+_, heatmap)
    #                 break
                    t_lay.append(res)

                    v3=outputs['relu']
                    t_r.append(res)
                    v3=torch.sum(v3,dim=1)
                    v3=np.squeeze(n_255(v3).cpu().numpy().astype(np.uint8))
                    if len(blanket2)<20:
                        sf2,res=layer_gain3(v3,sf2,2,2,10,0)
                    else :
                        sf2,res=layer_gain3(v3,sf2,2,1,5,0)
#                     get_G2(sf2,"relu","merge_")
#                     get_G3(sf2,listt1['relu'],"relu","show_")
#                     print("relu")
                    if len(blanket2)<20:
                        sf2,res=layer_gain3(tk,sf2,0,2,5,0)
                    else :
                        sf2,res=layer_gain3(tk,sf2,0,2,5,0)
#                     print(sf2.shape,listt1['relu'].shape)
#                     get_G2(sf2,"relu","merge_")
#                     get_G3(sf2,listt1['relu'],"relu","show_")
                    print()
                    c=tk.shape[0]
                    d=tk.shape[1]
                    sf3 = (cv2.resize(sf2, dsize=(d,c)))
                    ret,sf3=cv2.threshold(sf3,200,255,cv2.THRESH_BINARY)
                    cv2.imwrite("val_tanh_bce_nofc_c8/"+"_" +_, sf2)
                    sf2 = (cv2.resize(sf2, dsize=(d,c))*0.5).astype(np.int)
                    heatmap=np.zeros((c,d,3),np.uint8)
                    for i in range(c):
                        for j in range(d):
                            heatmap[i][j]=color_map[sf2[i][j]]

                    show_img = np.clip(heatmap + img, a_min=0, a_max=255).astype(np.uint8)
                    img = img.astype(np.uint8)
                    write_img = np.concatenate((img, show_img), axis=1)
                    write_img = cv2.putText(write_img, "target:" + str(tar) , (10, 60), 2,cv2.FONT_HERSHEY_PLAIN, (0, 0, 255))
                    cv2.imwrite("val_tanh_bce_nofc_c8/" +_, write_img)
#                     break
                    continue
                else:
#                     sf2=np.zeros((1024,512),np.uint8)
#                     cv2.imwrite("val_tanh_bce_nofc_c8/"+"_" +_, sf2)
                    continue
                    tp+="C"
                    print("CCCCC")
                    for i in range(c):
                        for j in range(d):
                          #  print(sf2[i][j])]
                            if feature_sum3[i][j]>10:
                                sf2[i][j]=max(feature_sum3[i][j],255)
#                     c=tk.shape[0]
#                     d=tk.shape[1]
#                     sf3 = (cv2.resize(sf2, dsize=(d,c)))
#                     ret,sf3=cv2.threshold(sf3,200,255,cv2.THRESH_BINARY)
#                     print(sf3.shape)
#                     print(sf3)
#                     print(_)
                   # cv2.imwrite("val_tanh_bce_nofc_c8/"+"_" +_, sf3)
                  #  continue
            c=feature_sum.shape[0]
            d=feature_sum.shape[1]
            for i in range(c):
                    for j in range(d):
                        heatmap[i][j]=color_map[sf2[i][j]]
            heatmap = (cv2.resize(heatmap, dsize=(img.shape[1], img.shape[0]))*0.5).astype(np.int)
            show_img = np.clip(img + heatmap, a_min=0, a_max=255).astype(np.uint8)
            img = img.astype(np.uint8)
            write_img = np.concatenate((img, show_img), axis=1)
            write_img = cv2.putText(write_img, "target:" + str(tar) , (10, 60), 2,cv2.FONT_HERSHEY_PLAIN, (0, 0, 255))
            cv2.imwrite("val_tanh_bce_nofc_c8/" +tp+_, write_img)
#             break


1024
-----------------------------------
0
0
0.5002168
-----------------------------------
2
0
0.5002257
-----------------------------------
4
1
0.99995863
6522_100.bmp
128 64
(128, 64)
151
151
BBBBB
1 73

-----------------------------------
6
1
0.9999796
6522_101.bmp
128 64
(128, 64)
145
145
BBBBB
1 77

-----------------------------------
8
1
0.99997926
6522_102.bmp
128 64
(128, 64)
145
145
BBBBB
1 73

-----------------------------------
10


KeyboardInterrupt: 

In [ ]:
# _9757_36.bmp

In [32]:
# def test():
#     try:
#         a=imageio.imread("_9757_36.bmp")
#     except FileNotFoundError:
#         a=np.zeros((1024,512),np.uint8)
# test()